In [ ]:
from glob import glob
import json

In [ ]:
all_chars = set(map(lambda x: x.split('\\')[-1][:-4], glob('data/wow_chars/*.txt')))
chars_with_comments = set(map(lambda x: x.split('\\')[-1][:-6], glob('data/char_comments/*.njson')))